In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import os
from tqdm import tqdm

In [4]:
student_id = ["M11107503", "M11010113"]
student_name = ["林璟芸", "張菁蘭"]

In [5]:
from fake_useragent import UserAgent
ua = UserAgent(use_cache_server=False)

In [6]:
sheet_names = [ 
                "家電・電玩・視聽・廚三機",
                "手機・平板・耳機・攝相機" ,
                "電腦・電競・週邊・零組件" ,
                "保健・醫療用品・保險套" ,
                "食品・零食・飲料・生鮮" ,
                "彩妝・保養・洗沐・香氛" ,
                "服裝・內衣褲・襪子・機能" ,
                "珠寶・精品・配飾・鞋包箱" ,
                "日用品・清潔・餐廚・寵物" ,
                "母嬰・童裝・尿布・玩具" ,
                "傢俱・寢具・傢飾・修繕" ,
                "運動・按摩・戶外・車類" ,
                "圖書・文具・影音・樂器" ,
                "旅遊・票券・加值・軟體"
                ]

In [7]:
query_df = pd.read_excel('query.xlsx', sheet_name=sheet_names)

In [8]:
query_df

{'家電・電玩・視聽・廚三機':    組別    搜尋詞
 0   3  可攜電磁爐
 1   3     暖氣
 2   3   大同電鍋
 3   3    延長線
 4   3    微波爐
 5   3    捕蚊燈
 6   3    吹風機
 7   3    掃地機
 8   3   免治馬桶
 9   3    除濕機,
 '手機・平板・耳機・攝相機':    組別        搜尋詞
 0   3       iPad
 1   3       三星手機
 2   3    ASUS 電腦
 3   3       行動電源
 4   3  Type C充電線
 5   3    iPad保護貼
 6   3       藍牙喇叭
 7   3      黑色碳粉匣
 8   3       智慧手錶
 9   3       攝影腳架,
 '電腦・電競・週邊・零組件':    組別          搜尋詞
 0   3     acer電競筆電
 1   3     asus電競電腦
 2   3     asus cpu
 3   3     TEAM SSD
 4   3      Acer 螢幕
 5   3         水冷風扇
 6   3           機殼
 7   3    FSP 電源供應器
 8   3          光碟機
 9   3  ViewSonic螢幕,
 '保健・醫療用品・保險套':    組別    搜尋詞
 0   3     護具
 1   3     輔具
 2   3   隱眼保液
 3   3   防蚊乳液
 4   3    OK繃
 5   3  隱形痘痘貼
 6   3    萬金油
 7   3    暖水袋
 8   3    洗吸鼻
 9   3    暖暖包,
 '食品・零食・飲料・生鮮':    組別     搜尋詞
 0   3  亞尼克生乳捲
 1   3    哈根達斯
 2   3    麥克雞塊
 3   3      炸物
 4   3    涼拌泡菜
 5   3      滷味
 6   3    黑糖饅頭
 7   3      果醋
 8   3    韭菜水餃
 9   3      罐頭,
 '彩妝・保養・洗沐・香氛':    組別     

In [9]:
pages = 5

In [10]:
product_information_columns = ['0', 'title', 'brand', 'link', 'price', 'amount', 'cate', 'desc']

In [11]:
ua = UserAgent()
headers = {'User-Agent': ua.random}

In [12]:
def force_request(url, timeout=5):
    try:
        resp = requests.get(url, headers=headers, timeout=timeout)
        return 0, resp
    except:
        if timeout > 60:
            return 1, []
        else:
            state, resp = force_request(url, timeout=timeout+5)        
            return state, resp

In [13]:
def get_query_urls(category, pages=5):
    urls = []
    category_df = query_df[category].head(10)
    keywords = category_df['搜尋詞'].tolist()
    for k in tqdm(range(len(keywords))):
        keyword = keywords[k]
        for page in range(1, pages+1):
            url = 'https://m.momoshop.com.tw/search.momo?_advFirst=N&_advCp=N&curPage={}&searchType=1&cateLevel=2&ent=k&searchKeyword={}&_advThreeHours=N&_isFuzzy=0&_imgSH=fourCardType'.format(page, keyword)

            state, resp = force_request(url)

            if state == 0:
                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, features="lxml")
                    for item in soup.select('li.goodsItemLi > a'):
                        item_url = 'https://m.momoshop.com.tw' + item['href']
                        if item_url not in urls:    urls.append(item_url)
                else:
                    print("沒有資料")
                    break
    return urls



In [14]:
def get_query_urls(category, pages=5):
    urls = []
    category_df = query_df[category].head(10)
    keywords = category_df['搜尋詞'].tolist()
    for k in tqdm(range(len(keywords))):
        keyword = keywords[k]
        for page in range(1, pages+1):
            url = 'https://m.momoshop.com.tw/search.momo?_advFirst=N&_advCp=N&curPage={}&searchType=1&cateLevel=2&ent=k&searchKeyword={}&_advThreeHours=N&_isFuzzy=0&_imgSH=fourCardType'.format(page, keyword)

            state, resp = force_request(url)

            if state == 0:
                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, features="lxml")
                    for item in soup.select('li.goodsItemLi > a'):
                        item_url = 'https://m.momoshop.com.tw' + item['href']
                        if item_url not in urls:    urls.append(item_url)
                else:
                    print("沒有資料")
                    break
    return urls

In [15]:
def get_product_information(soup):

    try:
        title = soup.find('meta',{'property':'og:title'})['content']
    except:
        title = ''

    try:
        brand = soup.find('meta',{'property':'product:brand'})['content']
    except:
        brand = ''

    try:
        link = soup.find('meta',{'property':'og:url'})['content']
    except:
        link = ''

    try:
        price = re.sub(r'\r\n| ','',soup.find('del').text)
    except:
        price = ''

    try:
        amount = soup.find('meta',{'property':'product:price:amount'})['content']
    except:
        amount = ''

    try:
        cate = ''.join([i.text for i in soup.findAll('article', {'class': 'pathArea'})])
        cate = re.sub('\n|\xa0', ' ', cate)
    except:
        cate = ''

    try:
        desc = soup.find('div', {'class': 'Area101'}).text
        desc = re.sub('\r|\n| ', '', desc)
    except:
        desc = 'Nan'

    return [title, brand, link, price, amount, cate, desc]



In [16]:
def get_product_extra_information(soup): 
    product_extra_information_json = {}
    columns = []
    values = []

    for i in soup.select('div.attributesArea > table > tr'):
        try:
            column = i.find('th').text
            column = re.sub('\n|\r| ', '', column)
            value = ''.join([j.text for j in i.findAll('li')])
            value = re.sub('\n|\r| ', '', value)

            if column not in product_extra_information_json.keys():
                product_extra_information_json[column] = [value]
            else:
                product_extra_information_json[column] = product_extra_information_json[column] + [value]

        except:
            pass
        
    for key in product_extra_information_json.keys():
        columns.append(key)
        values.append(product_extra_information_json[key][0])

    return values, columns



In [17]:
for sheet in sheet_names:
    print(sheet)
    time_out_json = {}
    df = pd.DataFrame(columns=product_information_columns)
    df_list = []
    timeout_df = pd.DataFrame(columns=['id', 'url'])
    sheet_name_clean = sheet.replace('・', '-')

    urls = get_query_urls(sheet)

    for u in tqdm(range(len(urls))):     
        url = urls[u]
        state, resp = force_request(url)
        if state == 0:
            soup = BeautifulSoup(resp.text, features="lxml")
            product_information = get_product_information(soup)
            product_extra_information, extra_columns = get_product_extra_information(soup)
            product_df = pd.DataFrame([[u]+product_information+product_extra_information], columns=product_information_columns+extra_columns)
            df_list.append(product_df)
        else:
            timeout_df.loc[len(timeout_df.index)] = [u, url]

    df = pd.concat(df_list, ignore_index=True)

    if not os.path.isdir(student_id[0]):   os.makedirs(student_id[0])
    df.to_excel(f'./{student_id[0]}/{student_id[0]}_{student_name[0]}_{sheet_name_clean}_hw_1.xlsx')

    if not os.path.isdir(student_id[1]):   os.makedirs(student_id[1])
    df.to_excel(f'./{student_id[1]}/{student_id[1]}_{student_name[1]}_{sheet_name_clean}_hw_1.xlsx')

    if not os.path.isdir('timeout'):   os.makedirs('timeout')
    timeout_df.to_excel(f'./timeout/timeout_{sheet_name_clean}.xlsx')

家電・電玩・視聽・廚三機


100%|█████████████████████████████████████████| 877/877 [15:52<00:00,  1.09s/it]


手機・平板・耳機・攝相機


100%|███████████████████████████████████████| 1000/1000 [13:57<00:00,  1.19it/s]


電腦・電競・週邊・零組件


...

100%|███████████████████████████████████████| 1000/1000 [21:50<00:00,  1.31s/it]


圖書・文具・影音・樂器


100%|█████████████████████████████████████████| 816/816 [17:54<00:00,  1.32s/it]


旅遊・票券・加值・軟體


100%|█████████████████████████████████████████| 663/663 [14:02<00:00,  1.27s/it]
